In [81]:
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
from datetime import datetime, timedelta
from utils import status_calc

In [4]:
START_DATE = "2010-01-01"
END_DATE = "2020-11-01"

In [114]:
def build_keystat():
    #Read raw data download from Quanel
    df_raw = pd.read_csv(r"SHARADAR_SF1.csv",index_col="calendardate")
    #Filter only quarterly statement
    is_ARQ = df_raw['dimension'] == 'ARQ'
    df_Quarter = df_raw[is_ARQ]

    #Select factors, drop any rows with Null value
    df_selected_factor = df_Quarter[['ticker',
        'eps','netinc','opinc','ncfo','cashnequsd',
        'de','ebitda','fcf','marketcap','netmargin',
        'pb','pe','ps','workingcapital','ev',
        'divyield','fcfps','revenue','invcap', 'ebit', 'equityusd', 'assets']]
    
    df_new = df_selected_factor.copy()
    df_new.dropna(axis=0, how="any", inplace=True)

    #Caclulate ratio
    df_ratio = df_new[['equityusd','netinc','assets','invcap','ebit']]
    df_roe= df_ratio['netinc'] / df_ratio['equityusd']
    df_roa = df_ratio['netinc'] / df_ratio['assets']
    df_roic = df_ratio['netinc'] / df_ratio['invcap']

    keystats_new = pd.concat([df_new, df_roe, df_roa, df_roic], axis =1)
    return keystats_new

In [90]:
def build_sp500_dataset(start=START_DATE, end=END_DATE):
    """
    Creates the dataset containing S&P500 prices
    :returns: sp500_index.csv
    """
    sp500_raw_data = pdr.get_data_yahoo("SPY", start=START_DATE, end=END_DATE)
    return sp500_raw_data

In [73]:
def build_stock_dataset(keystats_new = keystats_new, start=START_DATE, end=END_DATE):
    """
    Creates the dataset containing all stock prices
    :returns: stock_prices.csv
    """
    ticker_list = keystats_new.ticker.drop_duplicates().tolist()

    # Get all Adjusted Close prices for all the tickers in our list,
    # between START_DATE and END_DATE
    all_data = pdr.get_data_yahoo(ticker_list, start, end)
    stock_raw_data = all_data["Adj Close"]

    # Remove any columns that hold no data, and print their tickers.
    stock_raw_data.dropna(how="all", axis=1, inplace=True)
    missing_tickers = [
        ticker for ticker in ticker_list if ticker.upper() not in stock_raw_data.columns
    ]
    print(f"{len(missing_tickers)} tickers are missing: \n {missing_tickers} ")
    # If there are only some missing datapoints, forward fill.
    stock_raw_data.ffill(inplace=True)
    return stock_raw_data

In [78]:
def preprocess_price_data(sp500_raw_data, stock_raw_data):
    """
    Currently, the sp500 and stock price datasets we downloaded do not have any data for
    days when the market was closed (weekends and public holidays). We need to amend this so that
    all rows are included. Doing this now saves a lot of effort when we actually create the
    keystats dataset, which requires that we have stock data every day.
    :return: SP500 and stock dataframes, with no missing rows.
    """
    # Read in SP500 data and stock data, parsing the dates.
    #sp500_raw_data = pd.read_csv("sp500_index.csv", index_col="Date", parse_dates=True)
    #stock_raw_data = pd.read_csv("stock_prices.csv", index_col="Date", parse_dates=True)

    # We will reindex to include the weekends.
    start_date = str(stock_raw_data.index[0])
    end_date = str(stock_raw_data.index[-1])
    idx = pd.date_range(start_date, end_date)
    sp500_raw_data = sp500_raw_data.reindex(idx)
    stock_raw_data = stock_raw_data.reindex(idx)

    # Now the weekends are NaN, so we fill forward these NaNs
    # (i.e weekends take the value of Friday's adjusted close).
    sp500_raw_data.ffill(inplace=True)
    stock_raw_data.ffill(inplace=True)

    return sp500_raw_data, stock_raw_data

In [88]:
sp500_raw_data

In [96]:
def keystat_add_price(sp500_raw_data, stock_raw_data, keystats_new):
    #adding stock price with keystat

    stock_price = []
    sp500_price = []
    stock_1m_price = []
    sp500_1m_price = []
    stock_p_change = []
    sp500_p_change = []
    for i in range(keystats_new.shape[0]):
    #for i in range(100):    
        current_date = keystats_new.index[i]
        one_month_date = datetime.strptime(keystats_new.index[i],"%m/%d/%Y") + timedelta(weeks = 4)
        one_month_date = one_month_date.strftime("%m/%d/%Y")
        try:
            stock_price_i = float(stock_raw_data.loc[current_date, keystats_new.ticker[i]])
            stock_1m_price_i = float(stock_raw_data.loc[one_month_date, keystats_new.ticker[i]])
            sp500_price_i = float(sp500_raw_data.loc[current_date, "Adj Close"])
            sp500_1m_price_i = float(sp500_raw_data.loc[one_month_date, "Adj Close"])
            stock_p_change_i = round(
                    ((stock_1m_price_i - stock_price_i) / stock_price_i * 100), 2
            )
            sp500_p_change_i = round(
                    ((sp500_1m_price_i - sp500_price_i) / sp500_price_i * 100), 2
            )
        except KeyError:
            stock_price_i = None
            stock_1m_price_i = None
            sp500_price_i = None
            sp500_1m_price_i = None
            stock_p_change_i = None
            sp500_p_change_i = None
        stock_price.append(stock_price_i)
        stock_1m_price.append(stock_1m_price_i)
        sp500_price.append(sp500_price_i)
        sp500_1m_price.append(sp500_1m_price_i)   
        stock_p_change.append(stock_p_change_i)
        sp500_p_change.append(sp500_p_change_i)
    
    keystats_new["stock_price"] = stock_price
    keystats_new["sp500_price"] = sp500_price
    keystats_new["stock_p_change"] = stock_p_change
    keystats_new["sp500_p_change"] = sp500_p_change

    keystats_to_split = keystats_new.copy()
    keystats_to_split.dropna(axis=0, subset=["stock_price", "stock_p_change"], inplace=True)
    return keystats_to_split

In [111]:
def split_current_data(keystats_to_split):
    ticker_list = keystats_to_split.ticker.drop_duplicates().tolist()
    lastest_keystat = pd.DataFrame() 
    train_keystat = pd.DataFrame() 
    for i in ticker_list:
        try:
            lastest_i = keystats_to_split.loc[keystats_to_split["ticker"] == i].tail(1)
            train_i = keystats_to_split.loc[keystats_to_split["ticker"] == i].iloc[:-1]
        except:
            continue
    lastest_keystat = lastest_keystat.append(lastest_i)
    train_keystat = train_keystat.append(train_i)
    return lastest_keystat, train_keystat

In [115]:
build_keystat()
#stock_raw_data = build_stock_dataset()
#sp500_raw_data = build_sp500_dataset()
#sp500_df, stock_df = preprocess_price_data(sp500_raw_data, stock_raw_data)
#keystats_to_split = keystat_add_price(sp500_raw_data, stock_raw_data, keystats_new)
#lastest_keystat, train_keystat = split_current_data(keystats_to_split)

#train_keystat.to_csv("ketstats_to_train.csv", index = True)
#lastest_keystat.to_csv("forward_sample.csv", index = True)

,ticker,eps,netinc,opinc,ncfo,cashnequsd,de,ebitda,fcf,marketcap,...,divyield,fcfps,revenue,invcap,ebit,equityusd,assets,0,1,2
calendardate,,,,,,,,,,,,,,,,,,,,,
3/31/2010,A,0.31,108000000.0,154000000.0,225000000.0,4.209000e+09,1.957,197000000.0,197000000.0,1.047669e+10,...,0.0,0.566,1.271000e+09,2.973000e+09,161000000.0,2.624000e+09,7.767000e+09,0.041159,0.013905,0.036327
6/30/2010,A,0.59,205000000.0,115000000.0,90000000.0,3.879000e+09,2.245,311000000.0,63000000.0,1.004128e+10,...,0.0,0.182,1.384000e+09,4.070000e+09,251000000.0,2.802000e+09,9.100000e+09,0.073162,0.022527,0.050369
9/30/2010,A,0.84,292000000.0,203000000.0,373000000.0,4.205000e+09,2.001,337000000.0,339000000.0,1.412316e+10,...,0.0,0.985,1.576000e+09,4.149000e+09,270000000.0,3.228000e+09,9.696000e+09,0.090458,0.030116,0.070378
12/31/2010,A,0.56,193000000.0,211000000.0,120000000.0,2.655000e+09,1.412,284000000.0,82000000.0,1.589288e+10,...,0.0,0.236,1.519000e+09,4.207000e+09,221000000.0,3.332000e+09,8.044000e+09,0.057923,0.023993,0.045876
3/31/2011,A,0.58,200000000.0,266000000.0,378000000.0,2.975000e+09,1.186,344000000.0,327000000.0,1.660175e+10,...,0.0,0.942,1.677000e+09,4.178000e+09,280000000.0,3.953000e+09,8.649000e+09,0.050594,0.023124,0.047870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9/30/2020,CLW,1.29,21400000.0,49200000.0,55700000.0,4.750000e+07,2.682,71200000.0,46000000.0,6.566320e+08,...,0.0,2.772,4.574000e+08,2.334700e+09,43500000.0,4.962000e+08,1.826900e+09,0.043128,0.011714,0.009166
9/30/2020,AVNS,0.40,19300000.0,5200000.0,1100000.0,1.800000e+08,0.395,10800000.0,-1900000.0,1.928631e+09,...,0.0,-0.040,1.857000e+08,7.886000e+08,100000.0,1.289900e+09,1.799100e+09,0.014962,0.010728,0.024474
9/30/2020,ICD,-2.67,-15199000.0,-11676000.0,-1927000.0,1.881300e+07,0.573,-909000.0,-1046000.0,1.519251e+07,...,0.0,-0.183,1.022400e+07,5.663110e+08,-11676000.0,2.897210e+08,4.557170e+08,-0.052461,-0.033352,-0.026839


In [104]:
sp500_raw_data.shape

(2727, 6)

In [117]:
keystats_new.shape

(14770, 30)

In [107]:
keystats_to_split.shape

(14770, 30)

In [113]:
lastest_keystat.shape

(1, 30)